In [31]:
import argparse
import thulac
import json
import os

from tqdm import tqdm
from keras.preprocessing.text import Tokenizer
stop_words_path = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\NLP\\recomend_system\\Chinese\\hit_stopwords.txt'

datapath = 'C:\\Users\\luoyan011\\Desktop\\PersonalLearning\\GitHub\\GPT2-Chinese\\data'
vocab_size = 5000



## Process Text and Generate Vocabulary

In [36]:
import re
import pynlpir
def text_process(content):
    """
    功能：清洗微博内容并分词
    """
    processed_content = []
    # Replaces URLs with the word [URL]
    content = re.sub(r'(https?|ftp|file|www\.)[-A-Za-z0-9+&@#/%?=~_|!:,.;]+[-A-Za-z0-9+&@#/%=~_|]', '[URL]', content)
    # Replaces Email with the word [URL]
    content = re.sub(r'[a-zA-Z0-9_-]+@[a-zA-Z0-9_-]+[\.][a-zA-Z0-9_-]+', '[URL]', content)
    # Replaces user with the word FORWARD
    content = re.sub(r'(\/\/){0,1}@.*?(：|:| )', '[FORWARD]', content)
    # Replaces number  with the word [N]
    content = re.sub(r'\d+', '[N]', content)
    # Replace 2+ dots with space
    content = re.sub(r'[\.。…]{2,}', '。', content)
    # Replace 2+ ~~ 为 ~
    content = re.sub(r'~{2,}', '~', content)
    # Replace 2+ 叹号 为 一个叹号
    content = re.sub(r'[!！]{2,}', '!', content)
    # Replace 2+ 叹号 为 一个问号
    content = re.sub(r'[？?]{2,}', '?', content)
    # 去掉 //
    content = re.sub(r'//', ' ', content)
    # 去掉 引号
    content = re.sub(r'["“”\'‘’]', '', content)

    pynlpir.open(encoding='utf_8', encoding_errors='ignore')
    segments = pynlpir.segment(content, pos_tagging=False)
    i = 1
    count = len(segments) - 1
    for segment in segments:
        if re.match(r'\s+', segment):  # 过滤掉空格
            i = i + 1
            continue
        segment = re.sub(r'@[\S]+', '[USER_MENTION]', segment)
        processed_content.append(segment.strip())
        if (i == count) & (segment == '[USER_MENTION]'):  # 过滤掉最后一个单独的字
            break
        i = i + 1
    pynlpir.close()
    return processed_content


def datasetProcess(org_path,save_path,stop_words):
    """
    功能：过滤出微博内容重点中文并进行分词
    """
    outcome = []
    with open(org_path,"r",encoding="utf-8") as fp:
        for idx,item in enumerate(json.load(fp)):
            print("processing item {}".format(idx))
            content = item
            # content = "".join(regex.findall(chinese,content))
            seg_list = weibo_process(content)
            # seg_list = jieba.cut(content,cut_all=False)
            words = []
            for word in seg_list:
                if word in ignore_chars:
                    continue
                if word not in stop_words:
                    words.append(word)
            outcome.append(words)
    with open(save_path,"w",encoding="utf-8") as fp:
        json.dump(outcome,fp,ensure_ascii=False)

print('start read stopwords data.')
stopwords = []
with open(stop_words_path, 'r', encoding='utf-8') as f:
    for line in f:
        if len(line)>0:
            stopwords.append(line.strip())
print(stopwords)

start read stopwords data.
['———', '》），', '）÷（１－', '”，', '）、', '＝（', ':', '→', '℃', '&', '*', '一一', '~~~~', '’', '.', '『', '.一', './', '--', '』', '＝″', '【', '［＊］', '｝＞', '［⑤］］', '［①Ｄ］', 'ｃ］', 'ｎｇ昉', '＊', '//', '［', '］', '［②ｅ］', '［②ｇ］', '＝｛', '}', '，也', '‘', 'Ａ', '［①⑥］', '［②Ｂ］', '［①ａ］', '［④ａ］', '［①③］', '［③ｈ］', '③］', '１．', '－－', '［②ｂ］', '’‘', '×××', '［①⑧］', '０：２', '＝［', '［⑤ｂ］', '［②ｃ］', '［④ｂ］', '［②③］', '［③ａ］', '［④ｃ］', '［①⑤］', '［①⑦］', '［①ｇ］', '∈［', '［①⑨］', '［①④］', '［①ｃ］', '［②ｆ］', '［②⑧］', '［②①］', '［①Ｃ］', '［③ｃ］', '［③ｇ］', '［②⑤］', '［②②］', '一.', '［①ｈ］', '.数', '［］', '［①Ｂ］', '数/', '［①ｉ］', '［③ｅ］', '［①①］', '［④ｄ］', '［④ｅ］', '［③ｂ］', '［⑤ａ］', '［①Ａ］', '［②⑧］', '［②⑦］', '［①ｄ］', '［②ｊ］', '〕〔', '］［', '://', '′∈', '［②④', '［⑤ｅ］', '１２％', 'ｂ］', '...', '...................', '…………………………………………………③', 'ＺＸＦＩＴＬ', '［③Ｆ］', '」', '［①ｏ］', '］∧′＝［', '∪φ∈', '′｜', '｛－', '②ｃ', '｝', '［③①］', 'Ｒ．Ｌ．', '［①Ｅ］', 'Ψ', '－［＊］－', '↑', '.日', '［②ｄ］', '［②', '［②⑦］', '［②②］', '［③ｅ］', '［①ｉ］', '［①Ｂ］', '［①ｈ］', '［①ｄ］', '［①ｇ］', '［①②］', '［②ａ］', 'ｆ］', '

In [37]:
ignore_chars = ["/","@","【","】","#",":","[","]"]
datasetProcess(os.path.join(datapath, "train.json"),
               os.path.join(datapath, "train_process.json"),
               stopwords)

processing item 0
processing item 1
processing item 2
processing item 3
processing item 4
processing item 5
processing item 6
processing item 7
processing item 8
processing item 9
processing item 10
processing item 11
processing item 12
processing item 13
processing item 14
processing item 15
processing item 16
processing item 17
processing item 18
processing item 19
processing item 20
processing item 21
processing item 22
processing item 23
processing item 24
processing item 25
processing item 26
processing item 27
processing item 28
processing item 29
processing item 30
processing item 31
processing item 32
processing item 33
processing item 34
processing item 35
processing item 36
processing item 37
processing item 38
processing item 39
processing item 40
processing item 41
processing item 42
processing item 43
processing item 44
processing item 45
processing item 46
processing item 47
processing item 48
processing item 49
processing item 50
processing item 51
processing item 52
pro

processing item 416
processing item 417
processing item 418
processing item 419
processing item 420
processing item 421
processing item 422
processing item 423
processing item 424
processing item 425
processing item 426
processing item 427
processing item 428
processing item 429
processing item 430
processing item 431
processing item 432
processing item 433
processing item 434
processing item 435
processing item 436
processing item 437
processing item 438
processing item 439
processing item 440
processing item 441
processing item 442
processing item 443
processing item 444
processing item 445
processing item 446
processing item 447
processing item 448
processing item 449
processing item 450
processing item 451
processing item 452
processing item 453
processing item 454
processing item 455
processing item 456
processing item 457
processing item 458
processing item 459
processing item 460
processing item 461
processing item 462
processing item 463
processing item 464
processing item 465


In [38]:
f = open(os.path.join(datapath, 'train.json'), 'r', encoding = 'utf-8')
lines = json.load(f)
lines[0]

' 凌画喜欢海棠，特意买了一座山，种的全是海棠。  海棠盛开的最好时候，凌画在婢女亲卫的陪同下去栖云山赏海棠。  半路上，瞧见了一个长的十分好看的少年，少年一身华贵紫菱缎，骑着一匹青鬃马，拿着一把弯月弓，三箭齐发，三发齐中，三只奔跑中的梅花鹿纷纷中箭倒地，一众跑出来打猎的子弟们顿时满堂喝彩，齐齐叫了一声好。  少年收了弓箭，一脸的得意，眉眼轻扬，神采飞扬，本就精致好看的面孔一下子又增添了三分骄阳之色。  少年一捧清风艳，十里芝兰醉华庭。  凌画远远地瞧着，对身边人问，“那是谁？你可认识？”  琉璃一下子笑了，“那是端敬候府小侯爷，可惜了一手好箭法，不用在正道上。”  “哦？怎么说？”  琉璃笑道，“不愿去书院求学，也不愿去军中从军。说想做一辈子的纨绔，混吃等死好不快哉！”  凌画失笑，“是吗？”  “是啊！端敬候府的老侯爷和侯爷就是被他气死的。如今，端敬候府就他一个人了，感情好，没人管他了！”  凌画：“……”  她好像隐约听说过这事儿。  她盯着少年的脸看了片刻，吩咐，“你去问问，我有满山的海棠，让他赏三日，换他刚刚打的那三只梅花鹿，他可愿意？”  琉璃：“……”  她看着凌画，“小姐，您是认真的？”  “是啊，我想吃鹿肉。”  “三只都要？”  “都要！”  琉璃下了马车，招手带了几个侍卫，跟着她去了那一片猎场。  来到近前，琉璃对宴轻一福身，“宴小侯爷，您想看满山的海棠盛开吗？”  宴轻转过身，一脸的困惑。  琉璃伸手向远处一指，“就是那片山。”  宴轻更困惑了，但还是如实说，“挺想看的。据说那片山种了满山的海棠，盛开时，若红粉胭脂海，漂亮极了，可是主人从不对外开放，是以看到的人极少。我不认识那片山的主人，所以，不曾瞧过满山海棠盛开。”  琉璃抿嘴一笑，“那片山是我家小姐的，小姐正巧路过，想吃鹿肉，宴小侯爷若是将刚刚打的三只梅花鹿割舍的话，她就请您赏三日海棠。”  宴轻上下打量了琉璃一眼：“你家小姐？哪位？”  琉璃眨眨眼睛，没说话。  宴轻顿时警惕地后退了一步，“不会是什么阴谋吧？你家小姐是不是看我长的好看，想让我娶她？”  琉璃：“……”  她一脸无语，“我家小姐就是想吃鹿肉！宴小侯爷想多了！”  宴轻认真地辨认了一番琉璃的神色，见她不像作假，大松了一口气，“三只是不是有点儿多？你家小姐胃口可真大，不会是个胖子吧？”  琉璃深吸一口气，“您同意不同意？不同

In [39]:
f = open(os.path.join(datapath, 'train_process.json'), 'r', encoding = 'utf-8')
lines = json.load(f)
lines[0]

['凌',
 '画',
 '喜欢',
 '海棠',
 '特意',
 '买',
 '座',
 '山',
 '全',
 '海棠',
 '海棠',
 '盛开',
 '最',
 '好',
 '凌',
 '画',
 '婢女',
 '亲',
 '卫',
 '陪同',
 '下',
 '去',
 '栖云',
 '山',
 '赏',
 '海棠',
 '半路',
 '上',
 '瞧见',
 '长',
 '十分',
 '好看',
 '少年',
 '少年',
 '一身',
 '华贵',
 '紫',
 '菱',
 '缎',
 '骑',
 '匹',
 '青',
 '鬃',
 '马',
 '弯',
 '月',
 '弓',
 '三',
 '箭',
 '齐',
 '发',
 '三',
 '发',
 '齐',
 '中',
 '三',
 '只',
 '奔跑',
 '中',
 '梅花鹿',
 '纷纷',
 '中',
 '箭',
 '倒',
 '众',
 '跑',
 '出来',
 '打猎',
 '子弟',
 '顿时',
 '满堂喝彩',
 '齐齐',
 '声',
 '好',
 '少年',
 '收',
 '弓箭',
 '脸',
 '得意',
 '眉眼',
 '轻',
 '扬',
 '神采飞扬',
 '精致',
 '好看',
 '面孔',
 '一下子',
 '增添',
 '三分',
 '骄阳',
 '色',
 '少年',
 '捧',
 '清风',
 '艳',
 '十',
 '里',
 '芝兰',
 '醉',
 '华',
 '庭',
 '凌',
 '画',
 '远远地',
 '瞧',
 '身边',
 '人',
 '问',
 '认识',
 '琉璃',
 '一下子',
 '笑',
 '端',
 '敬',
 '候',
 '府',
 '小',
 '侯',
 '爷',
 '可惜',
 '手',
 '好',
 '箭',
 '法',
 '不',
 '正道',
 '上',
 '说',
 '琉璃',
 '笑',
 '道',
 '不',
 '愿',
 '去',
 '书院',
 '求学',
 '不',
 '愿',
 '去',
 '军中',
 '从军',
 '说',
 '想',
 '做',
 '一辈子',
 '纨',
 '绔',
 '混',
 '吃',
 '死',
 '好',
 '不',
 '快',
 '凌',
 '画',
 '失笑',

' 凌画喜欢海棠，特意买了一座山，种的全是海棠。  海棠盛开的最好时候，凌画在婢女亲卫的陪同下去栖云山赏海棠。  半路上，瞧见了一个长的十分好看的少年，少年一身华贵紫菱缎，骑着一匹青鬃马，拿着一把弯月弓，三箭齐发，三发齐中，三只奔跑中的梅花鹿纷纷中箭倒地，一众跑出来打猎的子弟们顿时满堂喝彩，齐齐叫了一声好。  少年收了弓箭，一脸的得意，眉眼轻扬，神采飞扬，本就精致好看的面孔一下子又增添了三分骄阳之色。  少年一捧清风艳，十里芝兰醉华庭。  凌画远远地瞧着，对身边人问，“那是谁？你可认识？”  琉璃一下子笑了，“那是端敬候府小侯爷，可惜了一手好箭法，不用在正道上。”  “哦？怎么说？”  琉璃笑道，“不愿去书院求学，也不愿去军中从军。说想做一辈子的纨绔，混吃等死好不快哉！”  凌画失笑，“是吗？”  “是啊！端敬候府的老侯爷和侯爷就是被他气死的。如今，端敬候府就他一个人了，感情好，没人管他了！”  凌画：“……”  她好像隐约听说过这事儿。  她盯着少年的脸看了片刻，吩咐，“你去问问，我有满山的海棠，让他赏三日，换他刚刚打的那三只梅花鹿，他可愿意？”  琉璃：“……”  她看着凌画，“小姐，您是认真的？”  “是啊，我想吃鹿肉。”  “三只都要？”  “都要！”  琉璃下了马车，招手带了几个侍卫，跟着她去了那一片猎场。  来到近前，琉璃对宴轻一福身，“宴小侯爷，您想看满山的海棠盛开吗？”  宴轻转过身，一脸的困惑。  琉璃伸手向远处一指，“就是那片山。”  宴轻更困惑了，但还是如实说，“挺想看的。据说那片山种了满山的海棠，盛开时，若红粉胭脂海，漂亮极了，可是主人从不对外开放，是以看到的人极少。我不认识那片山的主人，所以，不曾瞧过满山海棠盛开。”  琉璃抿嘴一笑，“那片山是我家小姐的，小姐正巧路过，想吃鹿肉，宴小侯爷若是将刚刚打的三只梅花鹿割舍的话，她就请您赏三日海棠。”  宴轻上下打量了琉璃一眼：“你家小姐？哪位？”  琉璃眨眨眼睛，没说话。  宴轻顿时警惕地后退了一步，“不会是什么阴谋吧？你家小姐是不是看我长的好看，想让我娶她？”  琉璃：“……”  她一脸无语，“我家小姐就是想吃鹿肉！宴小侯爷想多了！”  宴轻认真地辨认了一番琉璃的神色，见她不像作假，大松了一口气，“三只是不是有点儿多？你家小姐胃口可真大，不会是个胖子吧？”  琉璃深吸一口气，“您同意不同意？不同

In [41]:
def main():

    tokenizer = Tokenizer(num_words=vocab_size)

    f = open(os.path.join(datapath, 'train_process.json'), 'r', encoding = 'utf-8')
    lines = json.load(f)

    tokenizer.fit_on_texts(lines)
    vocab = list(tokenizer.index_word.values())
    pre = ['[SEP]', '[CLS]', '[MASK]', '[PAD]', '[UNK]']
    vocab = pre + vocab
    with open(os.path.join(datapath, 'vocab.txt'), 'w', encoding = 'utf-8') as f:
        for word in vocab[:vocab_size + 5]:
            f.write(word + '\n')


if __name__ == "__main__":
    main()